In [ ]:
import torch

In [ ]:
words = open("names.txt", "r").read().splitlines()

### E01.  Train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?*

##   1. Using counting

In [ ]:
# create a tensor which will hold the trigram counts
N = torch.ones((27, 27, 27), dtype=torch.int32) # use torch.ones instead of zeros for "smoothing" to avoid inf

In [ ]:
chars = sorted(list(set("".join(words)))) # create a sorted list of all characters
str_to_int = {s:i + 1 for i, s in enumerate(chars)} # create a mapping for characters to integers
str_to_int["."] = 0 # add a special start and end character
int_to_str = {i:s for s, i in str_to_int.items()} # create a reverse mapping for ints to characters


In [ ]:
# since we have a dictionary
# now we can create trigrams from the dataset (names.txt)

for w in words:
  chs = ["."] + list(w) + ["."]
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = str_to_int[ch1]
    ix2 = str_to_int[ch2]
    ix3 = str_to_int[ch3]
    N[ix1, ix2, ix3] += 1 # we add a count for each trigram in each word

P = (N).float() # transform the N tensor dtype from int to float
P /= P.sum(dim=2, keepdim=True) # normalize the P tensor to have a probability distribution that will equal to one for each slice of the tensor (P[0, 0].sum() == 1)
# ^ Reason why we use dim=2 is because we want to normalize the values for each "3rd" (index 2) dimension of the tensor:
# 0th dimension => rows
# 1st dimension => columns
# 2nd dimension => "slice" of the array (imagine a rubiks cube, the cube is like 3 2 dimensional arrays glued together)
# so we normalize each "slice" so that each row in each "slice" sums to 1

# In this step we:
# went through all the words,
# iterated through trigrams made from those words,
# and for every trigram we added a +1 to the N tensor which stores all of the counts for every possible trigram combination

In [ ]:
# This code generates 5 names using our probability distribution tensor P, starting at indexes 0
for i in range(5):
    out = []
    ix = 0
    jx = 0
    while True:
      p1 = P[ix][jx]
      # 0 0 5 P[0][0] -> n1
      # 0 5 x P[0][n1] -> n2
      # 5 x y P[n1][n2] -> n3

      next_letter = torch.multinomial(p1, 1, True).item()
      ix = jx # 0 -> 0
      jx = next_letter # 0 -> n1
      out.append(int_to_str[next_letter])
      if jx == 0:
        break

    print("".join(out))

kleel.
viceuximnie.
canahna.
ocorgitgoleilebettimaveazwggen.
luiree.


In [ ]:
# calculate loss (negative log likelihood)

log_likelihood = 0
n = 0
for w in words:
  chs = ["."] + list(w) + ["."]
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    # get the indexes for each trigram
    ix1, ix2, ix3 = str_to_int[ch1], str_to_int[ch2], str_to_int[ch3]
    # probability for current trigram
    prob = P[ix1, ix2, ix3]
    # log probability
    logprob = torch.log(prob)
    # log(a*b*c) = log(a) + log(b) + log(c)
    # add to log_likelihood (our loss)
    log_likelihood += logprob
    # add count for normalization
    n+= 1

print(f"{log_likelihood=}")
nll = -log_likelihood
print(f"{nll/n}") # <- average negative log likelihood (our loss function)

log_likelihood=tensor(-410414.9688)
2.092747449874878


# 2. Using NN

In [ ]:
import torch.nn.functional as F

In [ ]:
# ---- Summarized ----
# create the dataset
# initialize the weights
# do forward pass
# calculate loss
# backward propagation using gradient descent


def generate_dataset(words):
  xs1, ys = [], []
  for w in words:
    chs = ["."] + list(w) + ["."]
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
      ix1 = str_to_int[ch1]
      ix2 = str_to_int[ch2]
      ix3 = str_to_int[ch3]
      xs1.append([ix1, ix2])
      ys.append(ix3)

  xs1 = torch.tensor(xs1, dtype=torch.int64)
  ys = torch.tensor(ys, dtype=torch.int64)
  return xs1, ys

n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtrain, Ytrain = generate_dataset(words[:n1])
Xdev, Ydev = generate_dataset(words[n1:n2])
Xtest, Ytest = generate_dataset(words[n2:])

num = Xtrain.nelement()





In [ ]:
# create one-hot encodings for first two characters and concatenate them to allow us to pass them in as input to the net
Xenc = F.one_hot(Xtrain, num_classes=27).float()
Xenc = Xenc.view(-1, (27 * 2))
# xenc2 = F.one_hot(Xtrain2, num_classes=27).float()

# concat by the 1 dimension to have a tensor in which each row has 27*2 elements
# because we want to pass in 2 characters as input and get the prediction for the 3rd character
# xenc = torch.cat((xenc1, xenc2), 1)
Xenc.shape


torch.Size([157152, 54])

In [ ]:
print("number of examples: ", num)

number of examples:  314304


In [ ]:
# initialize weights
W = torch.randn((27*2, 27), requires_grad=True) # 27*2 matches the 2nd dimension of our input tensor (196113, 54) @ (54, 27) => (196113, 27)

In [ ]:
# gradient descent
for k in range(100):
  # this is the forward pass
  # logits = Xenc @ W # this is passing the input encodings into the neural net and multiplying by the weights
  logits = W[Xtrain[:, 0]] + W[Xtrain[:, 1] + 27] # instead of one hot encodings, we just index into the W matrix
  # (the reason we add 27 is because we have 54 rows in W, which correspond to 2 input characters so we add 27 to index into the second character embedding)
  loss = F.cross_entropy(logits, Ytrain) # we can use cross_entropy instead of calculating the loss manually, more efficient
  # counts = logits.exp() # this is getting the counts (equivalent to N tensor which had all the counts of character combinations)
  # probs = counts / counts.sum(1, keepdim=True) # normalizing the counts to get probabilities
  # loss = -probs[torch.arange(len(Xtrain)), Ytrain].log().mean() # calculating the nll loss
  loss += 0.0001*(W**2).mean()  # adding regularization
  # print(loss.item())


  # now we do the backward pass
  W.grad = None # zero out the gradient
  loss.backward() # this calculates the gradients for each parameter
  W.data += -1 * W.grad # update the parameters in the opposite direction of the gradients

  # the "-" is because if grad is positive that means that increasing the data will make the loss grow
  # so if grad is 0.123 and we update data by 0.01 * 0.123 the loss will grow
  # if we do -0.01 * 0.123 we will make data lower which will make the loss lower as well
  # if grad is -0.123 that means if data goes up the loss goes down
  # so we do -0.01 * -0.123 which makes data go up and the loss goes down

print(loss.item())

2.243472099304199


In [ ]:
# evaluate on dev set
Xdev_enc = F.one_hot(Xdev, num_classes=27).float()
Xdev_enc = Xdev_enc.view(-1, (27*2))
for k in range(100):
  logits = Xdev_enc @ W
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdim=True)
  dev_loss = -probs[torch.arange(len(Xdev)), Ydev].log().mean()

  W.grad = None
  dev_loss.backward()
  W.data += -1 * W.grad
print(dev_loss)

tensor(2.3925, grad_fn=<NegBackward0>)


In [ ]:
# evaluate on test set
Xtest_enc = F.one_hot(Xtest, num_classes=27).float()
Xtest_enc = Xtest_enc.view(-1, (27*2))
for k in range(1):
  logits = Xtest_enc @ W
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdim=True)
  test_loss = -probs[torch.arange(len(Xtest)), Ytest].log().mean()

print(test_loss)

tensor(2.4004, grad_fn=<NegBackward0>)


In [ ]:
# Now let's test the model by generating some names
# This will work similiarly to counting but with some changes
for i in range(5):
  out = []
  ix = 0
  jx = 0
  while True:
    # prepare inputs
    xenc1 = F.one_hot(torch.tensor([ix]), num_classes=27).float()
    xenc2 = F.one_hot(torch.tensor([jx]), num_classes=27).float()
    xenc = torch.cat((xenc1, xenc2), 1)
    # forward pass
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)
    # get the next letter
    next_letter = torch.multinomial(probs, 1, True).item()
    ix = jx
    jx = next_letter
    out.append(int_to_str[next_letter])
    if jx == 0:
      break
  print("".join(out))

urimle.
van.
caiyathyzerhses.
urnirn.
rle.
